In [ ]:
import json
from pathlib import Path
from typing import Optional,Any
from subdomains import sub_domains_search
import time




class subdomains_cache:

    def __init__(self,target_domain:str,):

        #캐시 파일 지정
        self.SUB_CACSHE_DIR = Path("data/cache/Subdomains")
        self.SUB_CACSHE_DIR.mkdir(parents=True,exist_ok=True)
        self.ttl = 30* 60 #  살아있는 시간 30분
        self.target_domain = target_domain

    def _now(self):
        return time.time()

    def _expired(self,save_at:int)->bool:

        if self._now - save_at > self.ttl:
            return True

        return False

    def _cache_save(self,subdomains:list[str])->bool:

        self.current_cache_dir = Path(f"{self.SUB_CACSHE_DIR}/{self.target_domain}.json")

        data_json = {
            "save_at": self._now,
            "domains": subdomains,
            #"ttl": self.ttl

        }
        save_data = json.dumps(list[subdomains],indent=2,ensure_ascii=False)
        self.current_cache_dir.write_text(save_data,encoding="utf-8")


    def _cache_load(self)->Optional[Any]:


        data =json.load(self.current_cache_dir.read_text(encoding="utf-8"))
        if not data:
            return None #아무것도 없으면 None


        if self._expired(data["save_at"]):
            return None #만료되면 None


        return data["domains"]

    def _load_data(self)->list[str]:

        domains_data = sub_domains_search(self.target_domain)
        return domains_data

    def _data_return_(self):
        update_domains_data = self._load_data()

        self._cache_save(update_domains_data)

        update_json = self._cache_load()

        return update_json["domains"]
    def discover(self):

        if self.current_cache_dir.exists():


            data = self._cache_load()

            if not data is None:
                return data["domains"]

            return self._data_return_()


        return self._data_return_()

    def __call__(self):
        return self.discover()




